**Avatars IDs**

- Mens US ASTM Sz M/40, MALE ID: "b4ec391f-4a63-41ad-8efd-f3e45c75bb28"

- Womens US ASTM Sz M/6, FEMALE ID: "ad80d4fd-cf03-4337-b42a-8df04ea07bcc"

# *Compute new positions of the garment*

In [285]:
import json
#IMPORT SCENE
rute = 'store_state_transformed.json'

with open(rute) as archivo:
    data = json.load(archivo)

# Avatar & Pieces geometry data
avatar = data["avatar"]
pieces = data["pieces"]
#print(pieces)
#print("Avatar", avatar)
all_pieces_indices = []
all_pieces_positon3d = []
for piece in data['pieces']:
    indices = piece['components']['garmentPiece']['geometry']['indices']
    posiciones = piece['components']['garmentPiece']['geometry']['position3D']
    all_pieces_indices.extend(indices)
    all_pieces_positon3d.extend(posiciones)

#print(len(all_pieces_positon3d))
#print("Pieces", len(all_pieces_positon3d))

In [286]:
import numpy as np

#Convert to numpy arrays & matrix
avatar_indices = np.array(avatar["indices"])
avatar_positions = np.array(avatar["positions"]).reshape(-1, 3)  # Convertir a una matriz de 3 columnas

pieces_indices = np.array(all_pieces_indices)
pieces_positions = np.array(all_pieces_positon3d).reshape(-1, 3)  # Convertir a una matriz de 3 columnas

print(len(pieces_positions))

8332


In [287]:
total_vertices_avatar = len(avatar_indices)
total_vertices_piezas = len(pieces_indices)

# Inicializar el arreglo de distancias minimas
distancias_minimas = np.zeros(len(pieces_positions))
ac = np.zeros(len(pieces_positions), dtype=int) #AvatarCorrespondeces
    # Iterar sobre los vértices de la prenda de ropa
for i, pos_prenda in enumerate(pieces_positions):
    distancias_vertice = np.linalg.norm(pos_prenda - avatar_positions, axis=1)
    
    # Encontrar el índice del vértice del avatar más cercano y su distancia mínima
    indice_vertice_cercano = np.argmin(distancias_vertice)
    distancia_minima = distancias_vertice[indice_vertice_cercano]
    
    # Almacenar la distancia mínima y el índice del vértice correspondiente en el avatar
    distancias_minimas[i] = distancia_minima
    ac[i] = indice_vertice_cercano

print("La distancia mínimas entre el avatar y la geometría de las piezas de ropa", distancias_minimas,ac)

La distancia mínimas entre el avatar y la geometría de las piezas de ropa [0.00770112 0.00987555 0.01133092 ... 0.06820608 0.01047858 0.03771483] [3723 3724 3769 ... 4854 4819 4267]


In [288]:
# Crear un array para almacenar los offsets entre cada vértice de la prenda y el vértice correspondiente del avatar
offsets = np.zeros_like(pieces_positions)

for i, indice_avatar in enumerate(ac):
    # Calcular el offset entre el vértice de la prenda y su vértice correspondiente en el avatar
    offset = np.array(avatar_positions[indice_avatar]) - np.array(pieces_positions[i])
    offsets[i]=offset

print(offsets)



[[ 0.00357375  0.00614297  0.00296638]
 [ 0.00512941  0.00769913  0.00345528]
 [ 0.00682111 -0.0083921   0.00338156]
 ...
 [-0.00154465  0.05604529 -0.0388408 ]
 [-0.00395495 -0.00856912  0.00455293]
 [ 0.00277418  0.0341382  -0.01578909]]


New Avatar

In [289]:
#IMPORT NEW AVATAR
avatars = 'avatars.json'

with open(avatars) as file:
    avatars_data = json.load(file)
new_avatar = avatars_data["male"]

new_avatar_positions= np.array(new_avatar["positions"]).reshape(-1, 3)  # Convertir a una matriz de 3 columnas


In [290]:
new_pieces_positions = np.zeros_like(pieces_positions)

for i, indice_avatar in enumerate(ac):
    # Calcular el offset entre el vértice de la prenda y su vértice correspondiente en el avatar
    new_pieces_positions[i] = np.array(new_avatar_positions[indice_avatar]) + np.array(offsets[i])


print(len(new_pieces_positions))


8332


In [291]:
import json
#IMPORT SCENE
rute = 'store_state_transformed.json'

with open(rute) as archivo:
    data = json.load(archivo)


pieces = data["pieces"]
new_list_posiciones = new_pieces_positions.reshape(-1).tolist()

for piece in data['pieces']:
    indices = piece['components']['garmentPiece']['geometry']['indices']
    posiciones = piece['components']['garmentPiece']['geometry']['position3D']

    # Determinar la longitud necesaria para la pieza actual
    longitud_pieza = len(posiciones)

    # Obtener las nuevas posiciones para la pieza actual
    nuevas_posiciones = new_list_posiciones[:longitud_pieza]

    # Reemplazar las posiciones originales por las nuevas posiciones
    piece['components']['garmentPiece']['geometry']['position3D'] = nuevas_posiciones
    
    # Actualizar new_pieces_positions para la próxima pieza
    new_list_posiciones = new_list_posiciones[longitud_pieza:]

with open("output.json", "w") as outfile: 
      json.dump(data, outfile)
print("Datos exportados a 'output.json'")

Datos exportados a 'output.json'
